<a href="https://colab.research.google.com/github/websitecreatr99/WOE_IV/blob/main/WOE_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import packages
import os
import pandas as pd
import numpy as np

df = pd.read_excel('bank.xlsx') #Read excel file

In [ ]:
df.head() 

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [ ]:
df['target'] = df['y'].apply(lambda x : 1 if x == 'yes' else 0)  # Convert to numeric
df = df.drop('y',axis=1)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd, numpy as np, os, re, math, time

# to check monotonicity of a series
def is_monotonic(temp_series):
    return all(temp_series[i] <= temp_series[i + 1] for i in range(len(temp_series) - 1)) or all(temp_series[i] >= temp_series[i + 1] for i in range(len(temp_series) - 1))

def prepare_bins(bin_data, c_i, target_col, max_bins):
    force_bin = True
    binned = False
    remarks = np.nan
    # ----------------- Monotonic binning -----------------
    for n_bins in range(max_bins, 2, -1):
        try:
            bin_data[c_i + "_bins"] = pd.qcut(bin_data[c_i], n_bins, duplicates="drop")
            monotonic_series = bin_data.groupby(c_i + "_bins")[target_col].mean().reset_index(drop=True)
            if is_monotonic(monotonic_series):
                force_bin = False
                binned = True
                remarks = "binned monotonically"
                break
        except:
            pass
    # ----------------- Force binning -----------------
    # creating 2 bins forcefully because 2 bins will always be monotonic
    if force_bin or (c_i + "_bins" in bin_data and bin_data[c_i + "_bins"].nunique() < 2):
        _min=bin_data[c_i].min()
        _mean=bin_data[c_i].mean()
        _max=bin_data[c_i].max()
        bin_data[c_i + "_bins"] = pd.cut(bin_data[c_i], [_min, _mean, _max], include_lowest=True)
        if bin_data[c_i + "_bins"].nunique() == 2:
            binned = True
            remarks = "binned forcefully"
    
    if binned:
        return c_i + "_bins", remarks, bin_data[[c_i, c_i+"_bins", target_col]].copy()
    else:
        remarks = "couldn't bin"
        return c_i, remarks, bin_data[[c_i, target_col]].copy()

# calculate WOE and IV for every group/bin/class for a provided feature
def iv_woe_4iter(binned_data, target_col, class_col):
    if "_bins" in class_col:
        binned_data[class_col] = binned_data[class_col].cat.add_categories(['Missing'])
        binned_data[class_col] = binned_data[class_col].fillna("Missing")
        temp_groupby = binned_data.groupby(class_col).agg({class_col.replace("_bins", ""):["min", "max"],
                                                           target_col: ["count", "sum", "mean"]}).reset_index()
    else:
        binned_data[class_col] = binned_data[class_col].fillna("Missing")
        temp_groupby = binned_data.groupby(class_col).agg({class_col:["first", "first"],
                                                           target_col: ["count", "sum", "mean"]}).reset_index()
    
    temp_groupby.columns = ["sample_class", "min_value", "max_value", "sample_count", "event_count", "event_rate"]
    temp_groupby["non_event_count"] = temp_groupby["sample_count"] - temp_groupby["event_count"]
    temp_groupby["non_event_rate"] = 1 - temp_groupby["event_rate"]
    temp_groupby = temp_groupby[["sample_class", "min_value", "max_value", "sample_count",
                                 "non_event_count", "non_event_rate", "event_count", "event_rate"]]
    
    if "_bins" not in class_col and "Missing" in temp_groupby["min_value"]:
        temp_groupby["min_value"] = temp_groupby["min_value"].replace({"Missing": np.nan})
        temp_groupby["max_value"] = temp_groupby["max_value"].replace({"Missing": np.nan})
    temp_groupby["feature"] = class_col
    if "_bins" in class_col:
        temp_groupby["sample_class_label"]=temp_groupby["sample_class"].replace({"Missing": np.nan}).astype('category').cat.codes.replace({-1: np.nan})
    else:
        temp_groupby["sample_class_label"]=np.nan
    temp_groupby = temp_groupby[["feature", "sample_class", "sample_class_label", "sample_count", "min_value", "max_value",
                                 "non_event_count", "non_event_rate", "event_count", "event_rate"]]
    
    """
    **********get distribution of good and bad
    """
    temp_groupby['distbn_non_event'] = temp_groupby["non_event_count"]/temp_groupby["non_event_count"].sum()
    temp_groupby['distbn_event'] = temp_groupby["event_count"]/temp_groupby["event_count"].sum()

    temp_groupby['woe'] = np.log(temp_groupby['distbn_non_event'] / temp_groupby['distbn_event'])
    temp_groupby['iv'] = (temp_groupby['distbn_non_event'] - temp_groupby['distbn_event']) * temp_groupby['woe']
    
    temp_groupby["woe"] = temp_groupby["woe"].replace([np.inf,-np.inf],0)
    temp_groupby["iv"] = temp_groupby["iv"].replace([np.inf,-np.inf],0)
    
    return temp_groupby

"""
- iterate over all features.
- calculate WOE & IV for there classes.
- append to one DataFrame woe_iv.
"""
def var_iter(data, target_col, max_bins):
    woe_iv = pd.DataFrame()
    remarks_list = []
    for c_i in data.columns:
        if c_i not in [target_col]:
            # check if binning is required. if yes, then prepare bins and calculate woe and iv.
            """
            ----logic---
            binning is done only when feature is continuous and non-binary.
            Note: Make sure dtype of continuous columns in dataframe is not object.
            """
            c_i_start_time=time.time()
            if np.issubdtype(data[c_i], np.number) and data[c_i].nunique() > 2:
                class_col, remarks, binned_data = prepare_bins(data[[c_i, target_col]].copy(), c_i, target_col, max_bins)
                agg_data = iv_woe_4iter(binned_data.copy(), target_col, class_col)
                remarks_list.append({"feature": c_i, "remarks": remarks})
            else:
                agg_data = iv_woe_4iter(data[[c_i, target_col]].copy(), target_col, c_i)
                remarks_list.append({"feature": c_i, "remarks": "categorical"})
            # print("---{} seconds. c_i: {}----".format(round(time.time() - c_i_start_time, 2), c_i))
            woe_iv = woe_iv.append(agg_data)
    return woe_iv, pd.DataFrame(remarks_list)

# after getting woe and iv for all classes of features calculate aggregated IV values for features.
def get_iv_woe(data, target_col, max_bins):
    func_start_time = time.time()
    woe_iv, binning_remarks = var_iter(data, target_col, max_bins)
    print("------------------IV and WOE calculated for individual groups.------------------")
    print("Total time elapsed: {} minutes".format(round((time.time() - func_start_time) / 60, 3)))
    
    woe_iv["feature"] = woe_iv["feature"].replace("_bins", "", regex=True)    
    woe_iv = woe_iv[["feature", "sample_class", "sample_class_label", "sample_count", "min_value", "max_value",
                     "non_event_count", "non_event_rate", "event_count", "event_rate", 'distbn_non_event',
                     'distbn_event', 'woe', 'iv']]
    
    iv = woe_iv.groupby("feature")[["iv"]].agg(["sum", "count"]).reset_index()
    print("------------------Aggregated IV values for features calculated.------------------")
    print("Total time elapsed: {} minutes".format(round((time.time() - func_start_time) / 60, 3)))
    
    iv.columns = ["feature", "iv", "number_of_classes"]
    null_percent_data=pd.DataFrame(data.isnull().mean()).reset_index()
    null_percent_data.columns=["feature", "feature_null_percent"]
    iv=iv.merge(null_percent_data, on="feature", how="left")
    print("------------------Null percent calculated in features.------------------")
    print("Total time elapsed: {} minutes".format(round((time.time() - func_start_time) / 60, 3)))
    iv = iv.merge(binning_remarks, on="feature", how="left")
    woe_iv = woe_iv.merge(iv[["feature", "iv", "remarks"]].rename(columns={"iv": "iv_sum"}), on="feature", how="left")
    print("------------------Binning remarks added and process is complete.------------------")
    print("Total time elapsed: {} minutes".format(round((time.time() - func_start_time) / 60, 3)))
    return iv, woe_iv.replace({"Missing": np.nan})

In [ ]:
iv, woe_iv = get_iv_woe(df.copy(), target_col="target", max_bins=20)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


------------------IV and WOE calculated for individual groups.------------------
Total time elapsed: 0.019 minutes
------------------Aggregated IV values for features calculated.------------------
Total time elapsed: 0.019 minutes
------------------Null percent calculated in features.------------------
Total time elapsed: 0.019 minutes
------------------Binning remarks added and process is complete.------------------
Total time elapsed: 0.02 minutes


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(iv)

      feature        iv  number_of_classes  feature_null_percent  \
0         age  0.003643                  2                   0.0   
1     balance  0.063773                  5                   0.0   
2    campaign  0.046811                  3                   0.0   
3     contact  0.247762                  3                   0.0   
4         day  0.002064                  2                   0.0   
5     default  0.000016                  2                   0.0   
6    duration  1.651501                  9                   0.0   
7   education  0.031812                  4                   0.0   
8     housing  0.106556                  2                   0.0   
9         job  0.132519                 12                   0.0   
10       loan  0.060791                  2                   0.0   
11    marital  0.040090                  3                   0.0   
12      month  0.379533                 12                   0.0   
13      pdays  0.008289                  2      

In [ ]:
print(woe_iv)

     feature     sample_class  sample_class_label  sample_count    min_value  \
0        age  (18.999, 41.17]                 0.0          2567         19.0   
1        age    (41.17, 87.0]                 1.0          1954         42.0   
2        age              NaN                 NaN             0          NaN   
3        job           admin.                 NaN           478       admin.   
4        job      blue-collar                 NaN           946  blue-collar   
..       ...              ...                 ...           ...          ...   
72  previous              NaN                 NaN             0          NaN   
73  poutcome          failure                 NaN           490      failure   
74  poutcome            other                 NaN           197        other   
75  poutcome          success                 NaN           129      success   
76  poutcome          unknown                 NaN          3705      unknown   

      max_value  non_event_count  non_e

In [ ]:
# # import packages
# import pandas as pd
# import numpy as np
# import pandas.core.algorithms as algos
# from pandas import Series
# import scipy.stats.stats as stats
# import re
# import traceback
# import string

# max_bin = 20
# force_bin = 3

# # define a binning function
# def mono_bin(Y, X, n = max_bin):
    
#     df1 = pd.DataFrame({"X": X, "Y": Y})
#     justmiss = df1[['X','Y']][df1.X.isnull()]
#     # print(justmiss)
#     notmiss = df1[['X','Y']][df1.X.notnull()]
#     # print(notmiss)
#     r = 0
#     while np.abs(r) < 1:
#         try:
#             d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
#             d2 = d1.groupby('Bucket', as_index=True)
#             r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
#             n = n - 1 
#         except Exception as e:
#             n = n - 1
            
#     # print(d1)
#     # print(d2)

#     if len(d2) == 1:
#         n = force_bin         
#         bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
#         if len(np.unique(bins)) == 2:
#             bins = np.insert(bins, 0, 1)
#             bins[1] = bins[1]-(bins[1]/2)
#         d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
#         d2 = d1.groupby('Bucket', as_index=True)
    
#     d3 = pd.DataFrame({},index=[])
#     print(d3)
#     d3["MIN_VALUE"] = d2.min().X
#     d3["MAX_VALUE"] = d2.max().X
#     d3["COUNT"] = d2.count().Y
#     d3["EVENT"] = d2.sum().Y
#     d3["NONEVENT"] = d2.count().Y - d2.sum().Y
#     d3=d3.reset_index(drop=True)
    
#     if len(justmiss.index) > 0:
#         d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
#         d4["MAX_VALUE"] = np.nan
#         d4["COUNT"] = justmiss.count().Y
#         d4["EVENT"] = justmiss.sum().Y
#         d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
#         d3 = d3.append(d4,ignore_index=True)
    
#     d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
#     d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
#     d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
#     d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
#     d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
#     d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
#     d3["VAR_NAME"] = "VAR"
#     d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
#     d3 = d3.replace([np.inf, -np.inf], 0)
#     d3.IV = d3.IV.sum()
    
#     return(d3)

# def char_bin(Y, X):
        
#     df1 = pd.DataFrame({"X": X, "Y": Y})
#     justmiss = df1[['X','Y']][df1.X.isnull()]
#     notmiss = df1[['X','Y']][df1.X.notnull()]    
#     df2 = notmiss.groupby('X',as_index=True)
    
#     d3 = pd.DataFrame({},index=[])
#     d3["COUNT"] = df2.count().Y
#     d3["MIN_VALUE"] = df2.sum().Y.index
#     d3["MAX_VALUE"] = d3["MIN_VALUE"]
#     d3["EVENT"] = df2.sum().Y
#     d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
#     if len(justmiss.index) > 0:
#         d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
#         d4["MAX_VALUE"] = np.nan
#         d4["COUNT"] = justmiss.count().Y
#         d4["EVENT"] = justmiss.sum().Y
#         d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
#         d3 = d3.append(d4,ignore_index=True)
    
#     d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
#     d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
#     d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
#     d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
#     d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
#     d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
#     d3["VAR_NAME"] = "VAR"
#     d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
#     d3 = d3.replace([np.inf, -np.inf], 0)
#     d3.IV = d3.IV.sum()
#     d3 = d3.reset_index(drop=True)
    
#     return(d3)

# def data_vars(df1, target):
    
#     stack = traceback.extract_stack()
#     filename, lineno, function_name, code = stack[-2]
#     vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
#     final = (re.findall(r"[\w']+", vars_name))[-1]
    
#     x = df1.dtypes.index
#     count = -1
    
#     for i in x:
#         if i.upper() not in (final.upper()):
#             if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
#                 conv = mono_bin(target, df1[i])
#                 conv["VAR_NAME"] = i
#                 count = count + 1
#             else:
#                 conv = char_bin(target, df1[i])
#                 conv["VAR_NAME"] = i            
#                 count = count + 1
                
#             if count == 0:
#                 iv_df = conv
#             else:
#                 iv_df = iv_df.append(conv,ignore_index=True)
    
#     iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
#     iv = iv.reset_index()
#     return(iv_df,iv) 

In [ ]:
# final_iv, IV = data_vars(df,df.target)